Written by Jair Lemmens
For educational use only

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

from main_modules import VQVAE, GeneticAlgorithm
from VAE_modules_20_11 import Decoder
from IPython.display import clear_output
from scipy.ndimage import gaussian_filter1d

import numpy as np
 
from PIL import Image
import math
from Tools.geometry_tools import draw_polygon, image_to_shapely, filter_polygons_size
from Tools.ladybug_tools import Climate
from Tools.sunlight_tools import shapely_shadow_matrix
import os
import shapely
import shapely.ops
from ladybug_geometry.triangulation import earcut
import PIL
import PIL.Image

from multiprocessing import Pool
import gradio as gr
import matplotlib

Written by Jair Lemmens
For educational use only

In [2]:
device = 'cuda:1'

In [3]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [4]:
# reference_graph = torch.tensor([
# [4,0,0,1,1,0],    """entry"""
# [0,12,1,0,0,1],   """kitchen"""
# [0,1,28,1,0,0],   """dining"""
# [1,0,1,28,0,0],   """living"""
# [1,0,0,0,2,0],   """toilet"""
# [0,1,0,0,0,4],    """washing"""
# ],device='cuda:0')

In [5]:
global reference_graphs
groundfloor_graph = torch.tensor(
    [
[10,0,0,1,1,1],    
[0,12,1,0,0,1],   
[0,1,28,1,0,0],   
[1,0,1,28,0,0],   
[1,0,0,0,2,0],   
[1,1,0,0,0,10],    
],device= device)
firstfloor_graph = torch.tensor([
[10,0,0],    
[0,12,1],   
[0,1,5],   
],device= device)


global colours

reference_graphs = [firstfloor_graph,groundfloor_graph]
target_areas = [graph.diagonal().sum() for graph in reference_graphs]
target_areas = torch.stack(list(reversed(target_areas))).to(torch.float)
colours = get_cmap(len(reference_graphs[0])+1)

global selected_floor
selected_floor = -1

In [14]:
renderer = nn.Sequential(Decoder(depths=[1,1,1,1,1,3],dims=[1,4,4,4,4,3]),nn.Sigmoid())
renderer.load_state_dict(torch.load('./trained_modules/nn.Sequential(Decoder(depths=[1,1,1,1,1,3],dims=[1,4,4,4,4,3]),nn.Sigmoid()).pt',map_location= device))
renderer = renderer.eval()

vqvae = VQVAE(device= device,dims=[1,32,16,2],codebook_size=16)
vqvae.load_state_dict(torch.load('trained_modules/VQVAE(dims=[1,32,16,2],codebook_size=16).pt',map_location= device))
vqvae.eval()
pass

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [10]:
files = os.listdir('./blender/toAI/')
floors = [PIL.Image.open(f'./blender/toAI/{file}').resize((512,512),PIL.Image.NEAREST) for file in reversed(files)]

In [11]:
amsterdam = Climate('./NLD_NH_Amsterdam-Schipol.AP.062400_TMYx.epw')
sky_vectors = amsterdam.sky_vectors
intensities = amsterdam.intensities

class Lighting_Criterium():
    def __init__(self,climate,target_irradiance=100):
        self.target_irradiance = target_irradiance
        self.sky_vectors = climate.sky_vectors
        self.intensities = torch.tensor(climate.intensities,dtype=torch.float32)
        self.total_irradiance = climate.intensities.sum()

    def score(self,multipoly):
        shadow_matrix = shapely_shadow_matrix(multipoly,self.sky_vectors)
        shadow = torch.einsum('i,ijk->jk',self.intensities,torch.tensor(shadow_matrix,dtype=torch.float32))
        light = self.total_irradiance-shadow
        mask = shadow_matrix[-1]
        sufficient = torch.where(light*mask>self.target_irradiance,1,0).sum()/mask.sum()
        return(sufficient.item())
    def light_levels(self,multipoly,target_z,floor_img):
        
        shadow_matrix = shapely_shadow_matrix(multipoly,self.sky_vectors,target_z=target_z)
        shadow = torch.einsum('i,ijk->jk',self.intensities,torch.tensor(shadow_matrix,dtype=torch.float32))
        light = self.total_irradiance-shadow
        light = light*floor_img
        area = floor_img.sum() 
        sufficient = torch.where(light*floor_img>self.target_irradiance,1,0).sum()/area
        return(torch.stack([1-(light.sum()/area)/self.total_irradiance,sufficient]))
    
    def shadows(self,multipoly,target_z,floor_img):
        shadow_matrix = shapely_shadow_matrix(multipoly,self.sky_vectors,target_z=target_z)
        shadow = torch.einsum('i,ijk->jk',self.intensities,torch.tensor(shadow_matrix,dtype=torch.float32))
        light = self.total_irradiance-shadow
        light = light*floor_img
        return(light)

light_crit = Lighting_Criterium(amsterdam)

In [12]:
def process_sample(sample):
    floors = sample[:2]

    floor_poly = []
    for n,layer in enumerate(floors):
        if n>0:    
            try:
                poly = image_to_shapely(layer*floors[n-1])
            except:
                poly = shapely.MultiPolygon(shapely.Polygon([]))
        else:
            poly = image_to_shapely(layer)

        poly = filter_polygons_size(poly,only_largest=True)        
        layer = np.zeros([32,32])
            
        floors[n] = torch.tensor(draw_polygon(poly,layer))
        
        floor_poly.extend(shapely.force_3d(poly,n).geoms)
        
    floor_poly = shapely.MultiPolygon(floor_poly)

    roofs = torch.max(sample[2:],floors)
    polygons = []
    for n,layer in enumerate(roofs):
        poly = image_to_shapely(layer)
        poly = filter_polygons_size(poly,only_largest=True)
        
        layer = np.zeros([32,32])        
        roofs[n] = torch.tensor(draw_polygon(poly,layer))           

        polygons.extend(shapely.force_3d(poly,n*3).geoms)
        polygons.extend(shapely.force_3d(poly,(n+1)*3).geoms)
    polygons = shapely.MultiPolygon(polygons)

    return(floors,roofs,polygons,floor_poly)

In [ ]:
poolsize = 100
steps = 15


genetic = GeneticAlgorithm(poolsize=poolsize,gene_shape=(8,8,16),num_parents=10,num_offspring=60,shift_rate=.10,replace_rate=.01)
for n in range(steps):
    with torch.no_grad():
        quant_out = vqvae.quantizer.get_indices(genetic.genes.argmax(dim=-1).to(vqvae.device))
        samples = vqvae.decoder(quant_out.reshape(-1,2,4,4)).sigmoid()
        samples= torch.where(samples>.5,1,0).reshape(poolsize,4,32,32)
        fitness = []

        samples = samples.cpu()
        for sample in samples:
            
            floors,roofs,polygons,floor_poly = process_sample(sample)
            
            areas = floors.sum([1,2])
            error = areas-target_areas*3
            error *=0.02

            area_score = (2/(math.e**error+math.e**-error)).mean()

            overhang_score = (floors.sum([1,2])/roofs.sum([1,2])).mean()

            compactness_score = 4*math.pi*polygons.area/(polygons.boundary.length**2)

            if area_score>.9 and overhang_score>.6 and compactness_score>.1:
                
                light_scores = torch.stack([light_crit.light_levels(polygons,0.01,floors[0]),light_crit.light_levels(polygons,3.01,floors[1])]).mean(-1)
                
            else:
                light_scores = torch.ones(2)*0.1*area_score*overhang_score*compactness_score
            
                
            fitness.append(light_scores)

               
    fitness = torch.stack(fitness).to(torch.float32)
    if n != steps-1:
        num = len(genetic.parent_indices(fitness))
        if num > 1:
            genetic.num_parents = num
        genetic.step(fitness)
    print(f'fitness = {fitness.max(dim=0)[0]} mean={fitness.mean(dim=0)},std={fitness.std(dim=0)}')        
    
# plt.scatter(fitness[:,0],fitness[:,1])



pareto_indices = genetic.parent_indices(fitness)

fig, axs = plt.subplots(2,len(pareto_indices))

for i,indice in enumerate(pareto_indices):

    floors,roofs,_,_ = process_sample(samples[indice])
    floors[1] = floors[1]*roofs[0]
    
    if i<5:
        for n,ax in enumerate(axs):
            roof = torch.stack([roofs[1-n],roofs[1-n],roofs[1-n]]).to(torch.float)
            floor = torch.stack([torch.zeros_like(roofs[1-n]),torch.zeros_like(roofs[1-n]),floors[1-n]]).to(torch.float)
            im = (np.clip((roof-floor).permute(1,2,0).numpy(),0,1)*255).astype(np.uint8)
            print(im.shape)
            Image.fromarray(im).resize((300,300),resample=Image.NEAREST).save(f'./p5/light_pareto/{i}.png')


In [ ]:
for file in os.listdir(f'p5/light_pareto/keep'):


    sample = Image.open(f'p5/light_pareto/keep/{file}')
    sample = sample.resize([32,32],resample=Image.NEAREST)
    sample = np.array(sample)[:,:,:3]

    roof = np.where(sample.sum(-1)>0,1,0)

    floor = np.where((sample.prod(-1)==0),roof,0)

    roof_poly = shapely.MultiPolygon([shapely.force_3d(image_to_shapely(roof),8).geoms])
    light = light_crit.shadows(roof_poly,0,floor)
    result = light
    result = light/amsterdam.intensities.sum()
    result = np.array((result*255)).astype(np.uint8)
    rad = light.sum()/floor.sum()/amsterdam.intensities.sum()
    Image.fromarray(result).resize((300,300),resample=Image.NEAREST).save(f'./p5/light_pareto/{rad}_1.png')



In [ ]:
# floors = torch.tensor(np.load('./designs/niemy_floors.npy'))
# roofs = torch.tensor(np.load('./designs/niemy_roofs.npy'))
# # polygons = []
# floors[1] = floors[1]*roofs[0]
    
# for n,floor in enumerate(floors):
#     roof = torch.stack([roofs[1-n],roofs[1-n],roofs[1-n]]).to(torch.float)
#     floor = torch.stack([torch.zeros_like(roofs[1-n]),torch.zeros_like(roofs[1-n]),floors[1-n]]).to(torch.float)
#     im = (np.clip((roof-floor).permute(1,2,0).numpy(),0,1)*255).astype(np.uint8)
#     print(im.shape)
#     Image.fromarray(im).resize((300,300),resample=Image.NEAREST).save(f'./p5/light_pareto/{i}.png')


# for n,layer in enumerate(torch.stack([floors,roofs]).flatten(0,1)): 
#     polygons.extend(shapely.force_3d(image_to_shapely(layer),n).geoms)
# polygons = shapely.MultiPolygon(polygons)
# tris = triangulate_polygon(polygons)
# tris_to_obj(tris,f'./designs/niemy.obj')

In [ ]:
# fig, axs = plt.subplots(2,1)


# for n,ax in enumerate(axs):
#     roof = torch.stack([roofs[1-n],roofs[1-n],roofs[1-n]]).to(torch.float)
#     floor = torch.stack([torch.zeros_like(roofs[1-n]),torch.zeros_like(roofs[1-n]),floors[1-n]]).to(torch.float)
    
#     ax.imshow((roof-floor).permute(1,2,0))
# plt.show

In [ ]:

def process(editor=None):
    global reference_graphs
    global mask
    global result
    global selected_floor

    reference_graph = reference_graphs[selected_floor]

    colours = get_cmap(len(reference_graph)+1)

    rooms_mask = np.zeros([len(reference_graph),32,32],np.float32)
    if editor !=None:
        
        drawing = editor['layers'][-1]
        drawing = np.array(drawing.resize((32,32),PIL.Image.NEAREST))[:,:,:3]

        temp_mask = np.where(drawing==[255,255,255],1,0).min(-1)
        
        if temp_mask.sum()>3:
            mask = torch.tensor(temp_mask,dtype=torch.float)
        for n in range(len(reference_graph)):
            rooms_mask[n] = np.where((abs(drawing-np.multiply(colours(n)[:3],255))<25).min(-1), 1., 0.)
    rooms_mask = torch.tensor(rooms_mask)

    with torch.no_grad():
        ones_conv = nn.Conv2d(len(reference_graph),len(reference_graph),kernel_size=5,stride=2,bias=False,groups=len(reference_graph))
        ones_conv.weight.fill_(1)

    B,N,C = 50,len(reference_graph),3

    num_parents = 5
    num_offspring = 35
    genetic = GeneticAlgorithm(num_parents=num_parents,num_offspring=num_offspring,poolsize=B,gene_shape=[N,C],shift_rate=.1,replace_rate=.1)


    results = []
    while True:
    
        claim = renderer(genetic.genes.reshape(B*N,C).unsqueeze(-1).unsqueeze(-1).sigmoid())
        
        claim = nn.functional.normalize(claim.reshape(B,N,*claim.shape[-2:]),dim=-3)

        sample = (claim*10000).softmax(-3)
        sample = torch.where(sample>.8,1.,0.)

        sample *= mask
        sample =  ((1-rooms_mask.sum(0).clamp(0,1))*sample)+rooms_mask
        
        intent = 1-((sample-claim)**2).mean((-3,-2,-1))

        adjacency_matrix = torch.zeros(B,len(reference_graph),len(reference_graph))
        stamps = ones_conv(sample).flatten(-2)
        
        for n in range(stamps.shape[-2]):
            adjacency_matrix[:,n] = torch.einsum("ij,ikj->ikj",torch.where(stamps[:,n]>3,1.0,0.0), stamps).sum(-1).clamp(0,1)
        
        adjacency_score = (1-(reference_graph.tril(-1)-adjacency_matrix.tril(-1)).clamp(0,1)**2).mean((-2,-1))
        areas = sample.sum((-2,-1))
        error = areas-reference_graph.diagonal()*3
        error *=.025
        area_score = (2/(math.e**error+math.e**-error)).mean(1)


        genetic.step(adjacency_score*area_score*intent)

        if adjacency_score[0].mean() > .99 and area_score[0].mean()>0.9:
            img = np.zeros((32,32,3))
            for n,layer in enumerate(sample[0]):
                img += np.einsum('k,ij->ijk',colours(n)[:3],np.where(layer.round()==1,1,0))
            results.append(PIL.Image.fromarray((img*255).astype(np.uint8)).resize((512,512),PIL.Image.NEAREST))

            genetic = GeneticAlgorithm(num_parents=num_parents,num_offspring=num_offspring,poolsize=B,gene_shape=[N,C],shift_rate=.1,replace_rate=.1)
        print(adjacency_score[0].item(),area_score[0].item())

        if len(results) == 5:
            break
    
    return results






def select_sample(editor,event_data: gr.SelectData):
    image = event_data.value['image']['path']
    image = PIL.Image.open(image)
    image.putalpha(100)
    temp = np.array(editor['layers'][-1])
    temp = np.einsum('ij,ij...->ij...',np.where((temp==(255,255,255,255)).min(-1),0,1),temp)
    editor['layers'][-1] = PIL.Image.fromarray((temp).astype(np.uint8))
    editor['background'] = image
    return(editor)


def select_mask(editor,event_data: gr.SelectData):
    global mask 
    global selected_floor
    selected_floor = event_data.index
    print(selected_floor)
    image = event_data.value['image']['path']
    image = PIL.Image.open(image)

    editor['layers'][-1] = image
    
    mask = np.array(image.resize((32,32),PIL.Image.NEAREST))[:,:,0]
    
    editor['background'] = image
    return(editor)



def lock(editor,floor_imgs):
    global selected_floor

    img = editor['background']
    img.putalpha(100)
    floor_imgs[selected_floor] = img
    
    return(floor_imgs)

brush_colors = [matplotlib.colors.rgb2hex(np.multiply(colours(n),[1,1,1,.1])) for n in range(len(reference_graphs[selected_floor]))]
brush_colors.insert(0,'#ffffff')

empty = floors[0]
start = {'background': empty, 'layers':[empty],'composite': None}



with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=20):
            editor = gr.ImageEditor(
                value = start,
                type="pil",
                image_mode='RGBA',
                interactive = True,
                brush=gr.Brush(default_size=30,colors=brush_colors),
                eraser=gr.Eraser(default_size=30)
            )
            
            button = gr.Button()
            
            gallery = gr.Gallery(interactive=False,scale=1, allow_preview=False,rows=0,selected_index=0,container=False,columns=5,object_fit='scale-down')
            lock_button = gr.Button(value='lock')
        with gr.Column():
            floor_imgs = gr.Gallery(interactive=True,scale=1, allow_preview=False,rows=2,selected_index=0,container=False,columns=1,object_fit='scale-down',value=reversed(floors))

    gallery.select(select_sample,editor,editor)
    floor_imgs.select(select_mask,editor,editor)

    button.click(process,editor,outputs=gallery)
    lock_button.click(lock,[editor,floor_imgs],outputs=floor_imgs)

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


0.8611111044883728 0.5278844237327576
0.8888888955116272 0.7829647064208984
0.9166666865348816 0.8922894597053528
0.9166666865348816 0.8922894597053528
0.8888888955116272 0.9484124183654785
0.8888888955116272 0.9484124183654785
0.9722222089767456 0.9048429131507874
0.9722222089767456 0.9048429131507874
0.9722222089767456 0.9048429131507874
0.9722222089767456 0.9048429131507874
0.9722222089767456 0.9048429131507874
0.9444444179534912 0.975351095199585
0.9444444179534912 0.9769117832183838
0.9444444179534912 0.9769117832183838
0.9444444179534912 0.9769117832183838
0.9444444179534912 0.9769117832183838
0.9444444179534912 0.9769117832183838
0.9444444179534912 0.9769117832183838
1.0 0.9753313660621643
0.8611111044883728 0.7700865268707275
0.9166666865348816 0.7895990014076233
0.9444444179534912 0.8617194294929504
0.9444444179534912 0.955956757068634
0.9444444179534912 0.955956757068634
0.9444444179534912 0.9617766737937927
0.9444444179534912 0.9617766737937927
0.9444444179534912 0.983418047